## Data Wrangling
Summary: 
- Separate train, val, test filepath in order to create neccessary FiftyOne dataset
- Extract label for each class_id for better data exploration (i.e. True breed name instead of numbers)

In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split

m_path = '/home/tungnguyendinh/.fastai/data/oxford-iiit-pet'
print(os.listdir(m_path))

['images', 'annotations']


In [2]:
import re
import json
df = pd.read_csv(f"{m_path}/annotations/list.txt", sep = " ", header=None, skiprows=6)
df.columns = ["file", "class", "type", "breed"]
df.head()
breed_regex = '([a-zA-Z_])+(?=_\d+)' #Regex to extract breed name from file name
df["breed_name"] = df["file"].apply(lambda x: re.search(breed_regex,x).group())
class_lst = list(df["breed_name"].unique())
class_lst
del df

In [3]:
trainval_df = pd.read_csv(f"{m_path}/annotations/trainval.txt", sep = " ", header=None)
trainval_df.columns = ["file", "class", "type", "breed"]

X = trainval_df["file"].apply(lambda x: f"{x}.jpg")
y = trainval_df["class"] - 1


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)      #Split into train val datasets
train_df = pd.concat([X_train, y_train], axis = 1)
val_df   = pd.concat([X_val, y_val], axis = 1)

test_df = pd.read_csv(f"{m_path}/annotations/test.txt", sep = " ", header=None)
test_df.columns = ["file", "class", "type", "breed"]
test_df["file"] = test_df["file"].apply(lambda x: f"{x}.jpg")
test_df["class"] = test_df["class"] - 1
test_df = test_df[["file", "class"]]

print(len(train_df))
print(len(val_df))
print(len(test_df))

2944
736
3669


In [4]:
#Convert to json file for FiftyOne

# train_dict = {"classes": class_lst}
# train_dict["labels"] = {x[1]["file"][:-4]:x[1]["class"] for x in train_df.iterrows()}
# train_json = json.dumps(train_dict, indent=4)
# with open("train.json", "w") as outfile:
#      outfile.write(train_json)
     
# val_dict   = {"classes": class_lst}
# val_dict["labels"] = {x[1]["file"][:-4]:x[1]["class"] for x in val_df.iterrows()}
# val_json = json.dumps(val_dict, indent=4)
# with open("val.json", "w") as outfile:
#      outfile.write(val_json)
     
# test_dict  = {"classes": class_lst}
# test_dict["labels"] = {x[1]["file"][:-4]:x[1]["class"] for x in test_df.iterrows()}
# test_json = json.dumps(test_dict, indent=4)
# with open("test.json", "w") as outfile:
#      outfile.write(test_json)
     


## Model building
Summary:
- Defined transformation class to augment data
- Finetune ResNet50 to breed classification problem
- Use FiftyOne to visualize results

In [5]:
import torch
from torchvision import transforms as T
from typing import Callable, Tuple, Union
from flash.core.data.transforms import ApplyToKeys
from flash.core.data.io.input_transform import InputTransform
from dataclasses import dataclass

@dataclass
class ICDTransform(InputTransform):
    image_size: Tuple[int, int] = (300, 300)
    mean: Union[float, Tuple[float, float, float]] = (0.485, 0.456, 0.406)
    std: Union[float, Tuple[float, float, float]] = (0.229, 0.224, 0.225)
    
    def per_sample_transform(self):
        return T.Compose([
            ApplyToKeys(
                "input",
                T.Compose([
                    T.ToTensor(),
                    T.Resize(self.image_size),
                    T.Normalize(self.mean, self.std),
                    T.RandomHorizontalFlip(),
                    T.ColorJitter(),
                    T.RandomPerspective()
                ])
            ),
            ApplyToKeys("target", torch.as_tensor)
        ])

/home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/pl_bolts/callbacks/data_monitor.py:20: UnderReviewWarning: The feature warn_missing_pkg is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  warn_missing_pkg("wandb")
/home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/pl_bolts/models/self_supervised/amdim/amdim_module.py:35: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API

In [6]:
from itertools import chain
import fiftyone as fo
from flash.core.classification import FiftyOneLabelsOutput as FOLO
from flash.image.classification.data import ImageClassificationData as ICD
from flash.core.integrations.fiftyone import visualize
from flash.image.classification.model import ImageClassifier as IC
from flash import Trainer

data_dir = f"{m_path}/images/"
BATCH_SIZE = 32

trainer = Trainer(
                       accelerator="gpu"
                     ,max_epochs=5
                     ,devices=[0]       #This argument refers to the cuda device
                     ,progress_bar_refresh_rate=1
                     ,auto_lr_find=True
                )


/home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:90: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=1)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
  rank_zero_deprecation(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [7]:
#NOTE: THIS CELL IS DEDICATED FOR FINETUNING RESNET50, IGNORE AND RUN NEXT CELL

train_dataset = fo.Dataset.from_dir(
  dataset_type=fo.types.FiftyOneImageClassificationDataset,
  data_path=data_dir,
  labels_path="./train.json"
)

val_dataset = fo.Dataset.from_dir(
  dataset_type=fo.types.FiftyOneImageClassificationDataset,
  data_path=data_dir,
  labels_path="./val.json"
)

test_dataset = fo.Dataset.from_dir(
  dataset_type=fo.types.FiftyOneImageClassificationDataset,
  data_path=data_dir,
  labels_path="./test.json"
)


datamodule = ICD.from_fiftyone(
  train_dataset=train_dataset,
  val_dataset=val_dataset,
  test_dataset=test_dataset,
  transform=ICDTransform,
  transform_kwargs=dict(image_size=(300, 300)),
  batch_size=BATCH_SIZE
)

model = IC(backbone='resnet50', num_classes=datamodule.num_classes)

                     
trainer.finetune(model, datamodule=datamodule, strategy=("freeze_unfreeze", 1))
trainer.save_checkpoint("ResNet50_LightningFLash_OxfordPet.pt")

 100% |███████████████| 2944/2944 [2.4s elapsed, 0s remaining, 1.3K samples/s]       
 100% |█████████████████| 736/736 [473.5ms elapsed, 0s remaining, 1.6K samples/s]      
 100% |███████████████| 3669/3669 [2.9s elapsed, 0s remaining, 1.2K samples/s]      


/home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/flash/core/data/utilities/paths.py:176: UserWarning: Found invalid file extensions: .mat. Files with these extensions will be ignored. The supported file extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp, .npy.
  rank_zero_warn(
/home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3508: FutureWarning: Please pass an instantiated object of the `InputTransform` class. Passing the Class and keyword arguments separately has been deprecated since v0.8.0 and will be removed in v0.9.0.
  exec(code_obj, self.user_global_ns, self.user_ns)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name          | Type           | Params
-------------------------------------------------
0 | train_metrics | ModuleDict     | 0     
1 | val_metrics   | ModuleDict     | 0     
2 | test_metrics  | ModuleDict     | 0     
3 | adapter       | DefaultAdapter | 23.6 M
-----------------------

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:110: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/torchvision/transforms/functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value is zero
  warnings.warn("Argument fill/fillcolor is not supported for Tensor input. Fill value is zero")


Validation sanity check:  50%|█████     | 1/2 [00:01<00:01,  1.43s/it]

/home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:56: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 32. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


/home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:110: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 1:   0%|          | 0/114 [00:00<?, ?it/s, loss=0.592, v_num=25, train_accuracy_step=0.719, train_cross_entropy_step=0.767, val_accuracy=0.908, val_cross_entropy=0.420]          

/home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/pytorch_lightning/callbacks/finetuning.py:207: UserWarning: The provided params to be frozen already exist within another group of this optimizer. Those parameters will be skipped.
HINT: Did you init your optimizer in `configure_optimizer` as such:
 <class 'torch.optim.adam.Adam'>(filter(lambda p: p.requires_grad, self.parameters()), ...) 
  rank_zero_warn(


Epoch 4: 100%|██████████| 114/114 [01:26<00:00,  1.31it/s, loss=0.941, v_num=25, train_accuracy_step=0.594, train_cross_entropy_step=1.230, val_accuracy=0.514, val_cross_entropy=1.850, train_accuracy_epoch=0.607, train_cross_entropy_epoch=1.250]


In [8]:
test_dataset = fo.Dataset.from_dir(
  dataset_type=fo.types.FiftyOneImageClassificationDataset,
  data_path=data_dir,
  labels_path="./test.json"
)

model = IC.load_from_checkpoint("ResNet50_LightningFLash_OxfordPet.pt")
datamodule = ICD.from_fiftyone(
  predict_dataset=test_dataset,
  batch_size = BATCH_SIZE
)

predictions = trainer.predict(model, datamodule=datamodule, output=FOLO(class_lst, return_filepath=False))
predictions = list(chain.from_iterable(predictions))
test_dataset.set_values("predictions", predictions)
results = test_dataset.evaluate_classifications("predictions", gt_field="ground_truth", eval_key="eval")

session = fo.launch_app(test_dataset, port=6008)    #Open localhost:6008 on any browser to interact with FiftyOne
session.wait()
# session.close()    #Execute this code to close FiftyOne session

 100% |███████████████| 3669/3669 [3.3s elapsed, 0s remaining, 1.0K samples/s]      


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:110: UserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 48 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting:  80%|████████  | 92/115 [00:00<00:06,  3.47it/s]

/home/datnguyenthanh/datnt_venv/lib/python3.8/site-packages/flash/core/classification.py:284: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pred = torch.tensor(pred)


Predicting: 100%|██████████| 115/115 [00:34<00:00,  1.45s/it]


Notebook sessions cannot wait


In [9]:
# session.close()